In [2]:
# Sample data

import numpy as np

FEE_HAIRCUT = 1.0 - 0.997
SWAP_CURVE_Y = np.arange(0.01, 1.0, 0.1 )
CHAIN_LIQUIDITY_TOK_1 = [2.0]*4
CHAIN_LIQUIDITY_TOK_2 = [1.0]*4


sample_swap_init_Y_str = "\tSwap curve Y: "
sample_chain_tok_1_str = "\tChain liquidity token 1:"
sample_chain_tok_2_str = "\tChain liquidity token 2:"
sample_fee_haircut_str = "\tFee haircut:"
print(f"Sample data \n"
      + sample_swap_init_Y_str + f"{SWAP_CURVE_Y} \n"
      + sample_chain_tok_1_str + f"{CHAIN_LIQUIDITY_TOK_1}\n"
      + sample_chain_tok_2_str + f"{CHAIN_LIQUIDITY_TOK_2}\n"
      + sample_fee_haircut_str + f"{FEE_HAIRCUT:6.4f}\n\n" )

Sample data 
	Swap curve Y: [0.01 0.11 0.21 0.31 0.41 0.51 0.61 0.71 0.81 0.91] 
	Chain liquidity token 1:[2.0, 2.0, 2.0, 2.0]
	Chain liquidity token 2:[1.0, 1.0, 1.0, 1.0]
	Fee haircut:0.0030




In [3]:
# Define forward and backward swap calculators
from typing import Union

def forward_swap(x:Union[float, np.ndarray], haircut:float = FEE_HAIRCUT) -> Union[float, np.ndarray]:
    if isinstance(x, float):
        assert x > 0
    elif isinstance(x, np.ndarray):
        assert np.all(x) > 0
    assert haircut > 0
    return haircut * x /(1 + haircut * x)

def backward_swap(y:Union[float, np.ndarray], haircut:float = FEE_HAIRCUT) -> Union[float, np.ndarray]:
    if isinstance(y, float):
        assert y > 0
    elif isinstance(y, np.ndarray):
        assert np.all(y) > 0
    assert haircut > 0
    return haircut * y /(1 - haircut * y)



In [4]:
# Define swap curves
#    Note: curves define the swap space, so haircut does not apply

x = backward_swap(SWAP_CURVE_Y, haircut=1.0)
y = forward_swap(x, haircut=1.0)

# Check if forward_swap equals inverse of backward_swap
inverse_test = np.allclose(y, SWAP_CURVE_Y)
print(f"Swap curves defined. \nForward swap is inverse of Backward swap: {inverse_test}")


Swap curves defined. 
Forward swap is inverse of Backward swap: True


In [20]:
print(x)

[1.01010101e-02 1.23595506e-01 2.65822785e-01 4.49275362e-01
 6.94915254e-01 1.04081633e+00 1.56410256e+00 2.44827586e+00
 4.26315789e+00 1.01111111e+01]


In [76]:
# Cyclic transaction calculations

def generate_exchange_data(in_leg_liq: np.ndarray, out_leg_liq: np.ndarray) -> list[dict]:
    assert len(in_leg_liq) == len(out_leg_liq)
    exchange_data = []
    for i in range(len(in_leg_liq)):
        exch = { "swap_number": i+1,
                 "liq_in": in_leg_liq[i],
                 "liq_out": out_leg_liq[i],}
        exchange_data.append(exch)
    return exchange_data

def generate_upper_bounds(exch_data: list[dict], x_curve: np.ndarray, y_curve: np.ndarray) -> np.ndarray:
    # Allocate eta array
    hat_eta = []

    # Calculate bounds

    # second swap
    i = 1
    exch_prior = exch_data[i]["swap_number"] - 1
    exch_current = exch_prior + 1
    print(f"Swap number {exch_data[i]['swap_number']}\n" )
    print(f"\t Prior exchange {exch_prior}" )
    print(f"\t Prior exchange outgoing leg liquidity: {exch_data[i]['liq_out']}")
    print(f"\t Current exchange {exch_current}" )
    print(f"\t Current exchange incoming leg liquidity: {exch_data[i]['liq_in']}")

    # a(i, i-1)/a(i,i+1)
    ratio_out_div_by_in = exch_data[exch_prior]["liq_out"] / exch_data[exch_current]["liq_in"]
    xi_indx = np.searchsorted(x_curve, ratio_out_div_by_in, side='right')

    print(f"\t Ratio {ratio_out_div_by_in}")
    print(f"\t X value above {x_curve[xi_indx]}")
    print(f"\t X value below {x_curve[xi_indx-1]}")

    eta = float(y_curve[xi_indx])
    eta_dict = {"exch_num": exch_current, "eta": eta, "y_value": float(y_curve[xi_indx])}
    print(f"\t Eta dict {eta_dict}")

    hat_eta.append(eta)

    for i in range(2,len(exch_data)-1):
        exch_prior = exch_data[i]["swap_number"] - 1
        exch_current = exch_prior + 1
        print(f"Swap number {exch_data[i]['swap_number']}\n" )
        print(f"\t Prior exchange {exch_prior}" )
        print(f"\t Prior exchange outgoing leg liquidity: {exch_data[i]['liq_out']}")
        print(f"\t Current exchange {exch_current}" )
        print(f"\t Current exchange incoming leg liquidity: {exch_data[i]['liq_in']}")

        # a(i, i-1)/a(i,i+1)
        ratio_out_div_by_in = exch_data[exch_prior]["liq_out"] / exch_data[exch_current]["liq_in"]
        xi_indx = np.searchsorted(x_curve, ratio_out_div_by_in, side='right')

        print(f"\t Ratio {ratio_out_div_by_in}")
        print(f"\t X value above {x_curve[xi_indx]}")
        print(f"\t X value below {x_curve[xi_indx-1]}")

        eta = float(y_curve[xi_indx]) * hat_eta[len(hat_eta)-1]
        hat_eta.append(eta)
        eta_dict = {"exch_num": exch_current, "eta": eta, "y_value": float(y_curve[xi_indx])}
        print(f"\t Eta dict {eta_dict}")

    return hat_eta





In [77]:
exch_cycle = generate_exchange_data(CHAIN_LIQUIDITY_TOK_1, CHAIN_LIQUIDITY_TOK_2)

In [78]:
print(exch_cycle)

[{'swap_number': 1, 'liq_in': 2.0, 'liq_out': 1.0}, {'swap_number': 2, 'liq_in': 2.0, 'liq_out': 1.0}, {'swap_number': 3, 'liq_in': 2.0, 'liq_out': 1.0}, {'swap_number': 4, 'liq_in': 2.0, 'liq_out': 1.0}]


In [79]:
upper_bounds = generate_upper_bounds(exch_data=exch_cycle, x_curve=x, y_curve=y)

Swap number 2

	 Prior exchange 1
	 Prior exchange outgoing leg liquidity: 1.0
	 Current exchange 2
	 Current exchange incoming leg liquidity: 2.0
	 Ratio 0.5
	 X value above 0.6949152542372882
	 X value below 0.4492753623188407
	 Eta dict {'exch_num': 2, 'eta': 0.41000000000000003, 'y_value': 0.41000000000000003}
Swap number 3

	 Prior exchange 2
	 Prior exchange outgoing leg liquidity: 1.0
	 Current exchange 3
	 Current exchange incoming leg liquidity: 2.0
	 Ratio 0.5
	 X value above 0.6949152542372882
	 X value below 0.4492753623188407
	 Eta dict {'exch_num': 3, 'eta': 0.16810000000000003, 'y_value': 0.41000000000000003}


In [80]:
print(upper_bounds)

[0.41000000000000003, 0.16810000000000003]


In [85]:
delta_1 = backward_swap(upper_bounds[0])
delta_1_prime =  forward_swap(upper_bounds[len(upper_bounds)-1])

In [86]:
print(delta_1)
print(delta_1_prime)

0.0012315147631586864
0.0005040458096981697
